# Introducing nanoHUB's SimTools #

*Steven Clark, Saaketh Desai, and Alejandro Strachan*

This notebook is a quick start guide to SimTools, nanoHUB new way of delivering online simulations.

SimTools in a nutshell

 * Define the inputs, outputs, and the compute required to generate outputs from inputs
   * Inputs can include ranges and units that are checked before running
 * SimTools can invoke parallel jobs running on HPC resources, a simple script, or evaluating a function call
 * The SimToolLib library validates inputs and outputs, including automatic unit conversion
 * Succesful runs are added to nanoHUB's simulation cache
 * If a user requests a previosuly executed run, we pull it from the cache with no delays and saving computing resources
 * SimTools can be invoked from graphical user interfaces (example here) or from a scientific/engineering script (example here)
 
The documentation for SimTools can be found [here](https://simtool.readthedocs.io/en/latest/).
 
This notebook demonstrates a simple SimTools that demonstrates all the possible input and output types.

## Step 1. Provide a description of your tool ##
This cell is optional but highly recommended. The provided description is displayed when returning SimTool search results.

In [ ]:
DESCRIPTION = """Show examples of SimTool input and output types"""

In [ ]:
%load_ext yamlmagic

## Step 2. Define all inputs. Including valid ranges and units is strongly encouraged ##

In [ ]:
%%yaml INPUTS

booleanValue:
    type: Boolean
    description: Execute bogus save operations
    value: False

textString:
    type: Text
    description: Text supplied as string
    value: textString

textFile:
    type: Text
    description: Text supplied as file

integerValue:
    type: Integer
    description: Simple integer
    value: 10
    
numberValue:
    type: Number
    description: Simple number
    value: 10.5
    min: 0.0
    max: 100.0
    units: K

# Specifying units clearly defines the units that the SimTool expects for the input. 
# Examples of inputs are K (for temperature), angstroms (for distance) and fs for time.
# If the input from the user is in some other units, we use the Pint library (https://pint.readthedocs.io/en/0.10.1/)
# to automatically convert the units, and, in combination with range checking, validating inputs for all SimTool runs.
# See the pint documentation for a list of allowable units.

arrayValue:
    type: Array
    description: Array of numbers
    value: [1,2,3]

arrayFile:
    type: Array
    description: Array as file
        
listValue:
    type: List
    description: List of values
    value: ['one','two','three']

listFile:
    type: List
    description: List as file

dictValue:
    type: Dict
    description: Dict of values
    value: {'one':1,'two':2,'three':3}

dictFile:
    type: Dict
    description: Dict as file

imageValue:
    type: Image
    description: Image as value
    
imageFile:
    type: Image
    description: Image as file
    
choiceValue:
    type: Choice
    description: Pick option from choices
    options: ['pear','apple','banana','grape','orange']
    value: pear

# Saaketh: let's describe how element works
# An input of type Element allows specification of a value such as 'Ni', in conjunction with a property, 
# which is then passed on to the Mendeleev library (https://mendeleev.readthedocs.io/en/stable/). 
# We use Mendeleev to get the values of the property automatically from their database. 
# This allows properties such as atomic_weight to specified with values as 'Ni' or 'Cu' instead of the raw number.

elementValue:
    type: Element
    description: Element (like in the periodic table) as value
    property: atomic_weight
    value: Ni

In [ ]:
# If you simulation require additional files (e.g. configuration files), list them here.  The files
# should exist in the same directory (simtool) as this notebook.
# This cell is optional.  The tag FILES and variable EXTRA_FILES must specified exactly as given here.
EXTRA_FILES = []

In [ ]:
# Values should be provided here for all input variables. The values should match those given in the INPUTS cell.
booleanValue = False
textString = "text string"
textFile = "text file"
integerValue = 10
numberValue = 10.5
arrayValue = [1,2,3]
arrayFile = []
listValue = ['one','two','three']
listFile  = []
dictValue = {'one':2,'two':2,'three':3}
dictFile = {}
imageValue = ""
imageFile = ""
choiceValue = "pear"
elementValue = "Ni"

In [ ]:
import sys
import os
import shutil
import numpy as np

from simtool import DB, parse

In [ ]:
%%yaml OUTPUTS

booleanValue:
    type: Boolean
    description: Execute bogus save operations
    value: False

textString:
    type: Text
    description: Text supplied as string
    value: textString

textFile:
    type: Text
    description: Text supplied as file

integerValue:
    type: Integer
    description: Simple integer
    value: 10

numberValue:
    type: Number
    description: Simple number
    value: 10.5
    min: 0.
    max: 100.

arrayValue:
    type: Array
    description: Array of numbers
    value: [1,2,3]

arrayFile:
    type: Array
    description: Array as file
        
listValue:
    type: List
    description: List of values
    value: ['one','two','three']

listFile:
    type: List
    description: List as file

dictValue:
    type: Dict
    description: Dict of values
    value: {'one':2,'two':2,'three':3}

dictFile:
    type: Dict
    description: Dict as file

imageValue:
    type: Image
    description: Image as value
    
imageFile:
    type: Image
    description: Image as file

choiceValue:
    type: Choice
    description: Pick option from choices
    options: ['pear','apple','banana','grape','orange']
    value: pear
    
elementValue:
    type: Element
    description: Element as value
    value: Ni

In [ ]:
db = DB(OUTPUTS)

In [ ]:
def copyAndSaveFileAsOutput(outputVariableName,inputPath):
    if inputPath.startswith('file://'):
        resultFile = os.path.basename(inputPath[7:])
        if resultFile != inputPath[7:]:
            shutil.copy2(inputPath[7:],resultFile)
    else:
        resultFile = os.path.basename(inputPath)
        if resultFile != inputPath:
            shutil.copy2(inputPath,resultFile)
    db.save(outputVariableName,file=resultFile)

In [ ]:
print(booleanValue)
db.save('booleanValue', booleanValue)

print(integerValue)
db.save('integerValue', integerValue)

print(numberValue)
db.save('numberValue', numberValue)

print(textString)
db.save('textString', textString)

print(textFile)
copyAndSaveFileAsOutput('textFile', textFile)

print(imageFile)
copyAndSaveFileAsOutput('imageFile', imageFile)

print(imageValue)
db.save('imageValue', imageValue)

print(arrayValue)
db.save('arrayValue', arrayValue)

print(arrayFile)
copyAndSaveFileAsOutput('arrayFile', arrayFile)

print(listValue)
db.save('listValue', listValue)

print(listFile)
copyAndSaveFileAsOutput('listFile', listFile)

print(dictValue)
db.save('dictValue', dictValue)

print(dictFile)
copyAndSaveFileAsOutput('dictFile', dictFile)

print(choiceValue)
db.save('choiceValue', choiceValue)

print(elementValue)
db.save('elementValue', elementValue)

In [ ]:
def copyAndSaveFileAsBogusOutput(outputVariableName,inputPath):
    if inputPath.startswith('file://'):
        resultFile = os.path.basename(inputPath[7:])
        if resultFile != inputPath[7:]:
            shutil.copy2(inputPath[7:],resultFile)
    else:
        resultFile = os.path.basename(inputPath)
        if resultFile != inputPath:
            shutil.copy2(inputPath,resultFile)
    try:
        db.save(outputVariableName,file=resultFile + "_bogus")
    except FileNotFoundError as e:
        print("%s" % (e.args[0]))

In [ ]:
if booleanValue:
    numberValue = -21.3
    print(numberValue)
    try:
        db.save('numberValue', numberValue)
    except ValueError as e:
        print(e.args[0])

    print(textFile)
    copyAndSaveFileAsBogusOutput('textFile', textFile)